In [1]:
# 1️⃣ Load Dataset & Split Data

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris

# Load dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split data: 80% training, 20% holdout
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set: {X_train.shape}, Holdout set: {X_holdout.shape}")


Training set: (120, 4), Holdout set: (30, 4)


In [3]:
# 2️⃣ Train Base Models

# Kita akan melatih dua model dasar:
# Random Forest
# K-Nearest Neighbors (KNN)

# Train base models
rf = RandomForestClassifier(n_estimators=100, random_state=42)
knn = KNeighborsClassifier(n_neighbors=5)

rf.fit(X_train, y_train)
knn.fit(X_train, y_train)

# Prediksi base models di holdout set
rf_preds = rf.predict_proba(X_holdout)  # Probabilitas kelas
knn_preds = knn.predict_proba(X_holdout)  # Probabilitas kelas

# Gabungkan prediksi dari base models sebagai fitur untuk meta model
meta_features = np.hstack((rf_preds, knn_preds))

print(f"Shape meta features: {meta_features.shape}")

# 🔎 Penjelasan:
# ✅ rf.predict_proba(X_holdout) → Menghasilkan probabilitas setiap kelas (bukan langsung label).
# ✅ np.hstack((rf_preds, knn_preds)) → Menggabungkan output kedua model jadi satu dataset baru.
# ✅ Meta features → Ini yang nanti bakal kita pakai buat melatih model meta.

# Kenapa (30, 6)?
# Kita punya 30 data di holdout set (karena 20% dari 150 data).
# 6 kolom karena:
# Random Forest (rf_preds) → 3 kolom (probabilitas untuk setiap kelas).
# KNN (knn_preds) → 3 kolom (probabilitas untuk setiap kelas).
# Total = 6 fitur buat meta model.

Shape meta features: (30, 6)


In [ ]:
# 3️⃣ Train Meta Model (Logistic Regression)

# Sekarang, kita gunakan meta_features buat melatih Logistic Regression sebagai meta model.

# Train meta model
meta_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=42)
meta_model.fit(meta_features, y_holdout)

# Prediksi akhir menggunakan meta model
final_preds = meta_model.predict(meta_features)

# Evaluasi model
accuracy = accuracy_score(y_holdout, final_preds)
print(f"Blending Accuracy: {accuracy:.4f}")

# 🔎 Penjelasan:
# ✅ meta_model.fit(meta_features, y_holdout) → Melatih Logistic Regression dengan prediksi dari base models.
# ✅ final_preds = meta_model.predict(meta_features) → Meta model membuat prediksi akhir.
# ✅ accuracy_score(y_holdout, final_preds) → Mengevaluasi akurasi dari blending model.

Blending Accuracy: 1.0000


c:\Users\formylife\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
